# Freq_meth_calculate usage

Calculate methylation frequency at genomic CpG sites from the output of `nanopolish call-methylation`

## Output files format

`Freq_meth_calculate` can generates 2 files, a standard BED file and a tabulated file containing extra information

#### BED file

Standard genomic BED6 (https://genome.ucsc.edu/FAQ/FAQformat.html#format1). The score correspond to the methylation frequency multiplied by 1000. The file is sorted by coordinates and can be rendered with a genome browser such as [IGV](https://software.broadinstitute.org/software/igv/)

#### Tabulated TSV file

Contrary to the bed file, in the tabulated report, positions are ordered by decreasing methylation frequency.

The file contains the following fields:

* **chrom / start / end / strand**: Genomic coordinates of the motif or group of motifs in case split_group was not selected.
* **site_id**: Unique integer identifier of the genomic position.
* **methylated_reads / unmethylated_reads / ambiguous_reads**: Number of reads at a given genomic location with a higher likelyhood of being methylated or unmethylated or with an ambiguous methylation call.
* **sequence**: -5 to +5 sequence of the motif or group of motifs in case split_group was not selected.
* **num_motifs**: Number of motif in the group.
* **meth_freq**: Methylation frequency (out of non anbiguous calls).

## Bash command line usage

### Command line help

In [1]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon NanopolishComp

NanopolishComp Freq_meth_calculate --help

usage: NanopolishComp Freq_meth_calculate [-h] [-i INPUT_FN]
                                          [-b OUTPUT_BED_FN]
                                          [-t OUTPUT_TSV_FN] [-d MIN_DEPTH]
                                          [-f FASTA_INDEX] [-s SAMPLE_ID]
                                          [-v | -q]

Calculate methylation frequency at genomic CpG sites from the output of
nanopolish call-methylation

optional arguments:
  -h, --help            show this help message and exit
  -v, --verbose         Increase verbosity (default: False)
  -q, --quiet           Reduce verbosity (default: False)

Input/Output options:
  -i INPUT_FN, --input_fn INPUT_FN
                        Path to a nanopolish call_methylation tsv output file.
                        If not specified read from std input
  -b OUTPUT_BED_FN, --output_bed_fn OUTPUT_BED_FN
                        Path to write a summary result file in BED format
  -t OUTPUT_TSV_FN, --output_tsv_fn OUTPUT_TSV_FN
        

### Example usage

#### From an existing nanopolish call_methylation file output

In [3]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon NanopolishComp

NanopolishComp Freq_meth_calculate --verbose -i data/freq_meth_calculate/methylation_calls.tsv -b ./output/freq_meth_calculate/out_freq_meth_calculate.bed -t ./output/freq_meth_calculate/out_freq_meth_calculate.tsv -s Sample1

head ./output/freq_meth_calculate/out_freq_meth_calculate.bed
head ./output/freq_meth_calculate/out_freq_meth_calculate.tsv

track name=Methylation_Sample1 itemRgb=On
chr-VIII	213351	213352	0	-0.500000	-	213351	213352	'100,100,100''
chr-VIII	213382	213386	1	-5.103000	-	213382	213386	'8,121,207''
chr-VIII	213427	213428	2	0.504545	-	213427	213428	'100,100,100''
chr-VIII	213516	213517	3	-2.447500	-	213516	213517	'8,121,207''
chr-VIII	213536	213537	4	-3.103333	-	213536	213537	'8,121,207''
chr-VIII	213549	213550	5	-2.724167	-	213549	213550	'8,121,207''
chr-VIII	213645	213646	6	-2.546667	-	213645	213646	'8,121,207''
chr-VIII	213668	213669	7	-3.728333	-	213668	213669	'8,121,207''
chr-VIII	213729	213730	8	-3.253846	-	213729	213730	'8,121,207''
chromosome	start	end	strand	site_id	methylated_reads	unmethylated_reads	ambiguous_reads	sequence	num_motifs	llr_list
chr-VIII	213351	213352	-	0	0	1	9	ACTAACGGGGA	1	-2.75,-0.46,1.48,-1.35,0.34,0.66,-1.58,-1.91,0.06,0.51
chr-VIII	213382	213386	-	1	1	7	2	TTCTTCGCCGACTG	2	-7.94,3.23,-0.77,0.05,-9.8,-5.7,-11.29,-5.61,-4.64,-8.56
chr-VIII	213427	213428	-	2	1	0	10	TTTCTCGCAAA	1	-0.28

## Options summary ##
	package_name: NanopolishComp
	package_version: 0.6.9
	timestamp: 2019-08-30 18:10:00.309729
	quiet: False
	verbose: True
	sample_id: Sample1
	min_depth: 10
	output_tsv_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.tsv
	output_bed_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.bed
	fasta_index: 
	input_fn: data/freq_meth_calculate/methylation_calls.tsv
## Checking arguments ##
	Testing input file readability
	Check output file
		Output results in bed format
		Output results in tsv format
## Parsing methylation_calls file ##
	Write output file header
	Starting to parse file Nanopolish methylation call file
	: 605248 lines [00:14, 42479.54 lines/s]
	Filtering out low coverage sites
	Processing valid sites found
	: 100%|██████████| 999/999 [00:01<00:00, 700.68 sites/s]
## Results summary ##
	Total read lines: 605,248
	Valid read lines: 605,248
	Total sites: 340,081
	Low coverage sites: 339,082
	Valid sites: 1,998


#### Using a fasta index for output coordinates sorting

In [5]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon NanopolishComp

NanopolishComp Freq_meth_calculate --verbose -i data/freq_meth_calculate/methylation_calls.tsv -b ./output/freq_meth_calculate/out_freq_meth_calculate.bed -t ./output/freq_meth_calculate/out_freq_meth_calculate.tsv -s Sample1 -f data/freq_meth_calculate/ref.fa.fai

head ./output/freq_meth_calculate/out_freq_meth_calculate.bed
head ./output/freq_meth_calculate/out_freq_meth_calculate.tsv

track name=Methylation_Sample1 itemRgb=On
chr-VIII	213351	213352	0	-0.500000	-	213351	213352	'100,100,100''
chr-VIII	213382	213386	1	-5.103000	-	213382	213386	'8,121,207''
chr-VIII	213427	213428	2	0.504545	-	213427	213428	'100,100,100''
chr-VIII	213516	213517	3	-2.447500	-	213516	213517	'8,121,207''
chr-VIII	213536	213537	4	-3.103333	-	213536	213537	'8,121,207''
chr-VIII	213549	213550	5	-2.724167	-	213549	213550	'8,121,207''
chr-VIII	213645	213646	6	-2.546667	-	213645	213646	'8,121,207''
chr-VIII	213668	213669	7	-3.728333	-	213668	213669	'8,121,207''
chr-VIII	213729	213730	8	-3.253846	-	213729	213730	'8,121,207''
chromosome	start	end	strand	site_id	methylated_reads	unmethylated_reads	ambiguous_reads	sequence	num_motifs	llr_list
chr-VIII	213351	213352	-	0	0	1	9	ACTAACGGGGA	1	-2.75,-0.46,1.48,-1.35,0.34,0.66,-1.58,-1.91,0.06,0.51
chr-VIII	213382	213386	-	1	1	7	2	TTCTTCGCCGACTG	2	-7.94,3.23,-0.77,0.05,-9.8,-5.7,-11.29,-5.61,-4.64,-8.56
chr-VIII	213427	213428	-	2	1	0	10	TTTCTCGCAAA	1	-0.28

## Options summary ##
	package_name: NanopolishComp
	package_version: 0.6.9
	timestamp: 2019-08-30 18:11:54.197782
	quiet: False
	verbose: True
	sample_id: Sample1
	min_depth: 10
	output_tsv_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.tsv
	output_bed_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.bed
	fasta_index: data/freq_meth_calculate/ref.fa.fai
	input_fn: data/freq_meth_calculate/methylation_calls.tsv
## Checking arguments ##
	Testing input file readability
	Check output file
		Output results in bed format
		Output results in tsv format
## Parsing methylation_calls file ##
	Write output file header
	Starting to parse file Nanopolish methylation call file
	: 605248 lines [00:13, 43975.59 lines/s]
	Filtering out low coverage sites
	Sorting by coordinates
	Processing valid sites found
	: 100%|██████████| 999/999 [00:01<00:00, 687.78 sites/s]
## Results summary ##
	Total read lines: 605,248
	Valid read lines: 605,248
	Total sites: 340,081
	Low coverage sites: 33

#### Changing filtering threshold (not recommended)

In [7]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon NanopolishComp

NanopolishComp Freq_meth_calculate --verbose -i data/freq_meth_calculate/methylation_calls.tsv -b ./output/freq_meth_calculate/out_freq_meth_calculate.bed -t ./output/freq_meth_calculate/out_freq_meth_calculate.tsv --min_depth 5

head ./output/freq_meth_calculate/out_freq_meth_calculate.bed
head ./output/freq_meth_calculate/out_freq_meth_calculate.tsv

track name=Methylation_ itemRgb=On
chr-I	68274	68275	0	-3.016000	-	68274	68275	'8,121,207''
chr-I	68295	68296	1	-3.200000	-	68295	68296	'8,121,207''
chr-I	68306	68311	2	-3.716000	-	68306	68311	'8,121,207''
chr-I	88983	88984	3	-2.148333	-	88983	88984	'8,121,207''
chr-I	89003	89004	4	-2.950000	-	89003	89004	'8,121,207''
chr-I	89023	89024	5	-3.480000	-	89023	89024	'8,121,207''
chr-I	89092	89093	6	-3.578333	-	89092	89093	'8,121,207''
chr-I	89105	89106	7	-1.385000	-	89105	89106	'100,100,100''
chr-I	89273	89274	8	-3.181667	-	89273	89274	'8,121,207''
chromosome	start	end	strand	site_id	methylated_reads	unmethylated_reads	ambiguous_reads	sequence	num_motifs	llr_list
chr-I	68274	68275	-	0	0	4	1	GGGACCGATCT	1	-1.1,-2.49,-5.42,-3.41,-2.66
chr-I	68295	68296	-	1	0	3	2	ACTTTCGGTAT	1	-1.34,-5.41,-0.14,-6.11,-3.0
chr-I	68306	68311	-	2	0	3	2	CATCACGGTCGCAGG	2	-2.89,-2.95,-9.29,-1.75,-1.7
chr-I	88983	88984	-	3	0	3	3	CTCTACGAATT	1	0.02,-1.06,-4.97,-3.01,-0.74,-3.13
chr-I	89003	89004	-	4	1	4	1	AGCTTCGGTTT

## Options summary ##
	package_name: NanopolishComp
	package_version: 0.6.9
	timestamp: 2019-08-30 18:12:46.143103
	quiet: False
	verbose: True
	sample_id: 
	min_depth: 5
	output_tsv_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.tsv
	output_bed_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.bed
	fasta_index: 
	input_fn: data/freq_meth_calculate/methylation_calls.tsv
## Checking arguments ##
	Testing input file readability
	Check output file
		Output results in bed format
		Output results in tsv format
## Parsing methylation_calls file ##
	Write output file header
	Starting to parse file Nanopolish methylation call file
	: 605248 lines [00:14, 43000.69 lines/s]
	Filtering out low coverage sites
	Processing valid sites found
	: 100%|██████████| 4544/4544 [00:02<00:00, 2052.97 sites/s]
## Results summary ##
	Total read lines: 605,248
	Valid read lines: 605,248
	Total sites: 340,081
	Low coverage sites: 335,537
	Valid sites: 9,088


## Python API usage

### Import the package

In [8]:
# Import main program
from NanopolishComp.Freq_meth_calculate import Freq_meth_calculate

# Import helper functions
from NanopolishComp.common import jhelp, head

### python API help

In [9]:
jhelp(Freq_meth_calculate)

---

**NanopolishComp.Freq_meth_calculate.__init__**

Calculate methylation frequency at genomic CpG sites from the output of nanopolish call-methylation

---

* **input_fn** *: str (required)*

Path to a nanopolish call_methylation tsv output file

* **fasta_index** *: str (default = )*

fasta index file obtained with samtools faidx. Required for coordinate sorting

* **output_bed_fn** *: str (default = )*

Path to write a summary result file in BED format

* **output_tsv_fn** *: str (default = )*

Path to write an more extensive result report in TSV format

* **min_depth** *: int (default = 10)*

Minimal number of reads covering a site to be reported

* **sample_id** *: str (default = )*

Sample ID to be used for the bed track header

* **verbose** *: bool (default = False)*

Increase verbosity

* **quiet** *: bool (default = False)*

Reduce verbosity



### Example usage

#### basic setting

In [10]:
f = Freq_meth_calculate(
    input_fn="./data/freq_meth_calculate/methylation_calls.tsv",
    output_bed_fn="./output/freq_meth_calculate/out_freq_meth_calculate.bed",
    sample_id="Sample1",
    verbose=True)

head("./output/freq_meth_calculate/out_freq_meth_calculate.bed")

## Options summary ##
	package_name: NanopolishComp
	package_version: 0.6.9
	timestamp: 2019-08-30 18:13:27.895308
	quiet: False
	verbose: True
	sample_id: Sample1
	min_depth: 10
	output_tsv_fn: 
	output_bed_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.bed
	fasta_index: 
	input_fn: ./data/freq_meth_calculate/methylation_calls.tsv
## Checking arguments ##
	Testing input file readability
	Check output file
		Output results in bed format
## Parsing methylation_calls file ##
	Write output file header
	Starting to parse file Nanopolish methylation call file
	: 605248 lines [00:16, 37771.09 lines/s]
	Filtering out low coverage sites
	Processing valid sites found
	: 100%|██████████| 999/999 [00:01<00:00, 622.77 sites/s] 
## Results summary ##
	Total read lines: 605,248
	Valid read lines: 605,248
	Total sites: 340,081
	Low coverage sites: 339,082
	Valid sites: 1,998


track name=Methylation_Sample1 itemRgb=On
chr-VIII	213351	213352	0	-0.500000	-	213351	213352	'100,100,100''
chr-VIII	213382	213386	1	-5.103000	-	213382	213386	'8,121,207''
chr-VIII	213427	213428	2	0.504545	-	213427	213428	'100,100,100''
chr-VIII	213516	213517	3	-2.447500	-	213516	213517	'8,121,207''
chr-VIII	213536	213537	4	-3.103333	-	213536	213537	'8,121,207''
chr-VIII	213549	213550	5	-2.724167	-	213549	213550	'8,121,207''
chr-VIII	213645	213646	6	-2.546667	-	213645	213646	'8,121,207''
chr-VIII	213668	213669	7	-3.728333	-	213668	213669	'8,121,207''
chr-VIII	213729	213730	8	-3.253846	-	213729	213730	'8,121,207''



#### Using a fasta index for output coordinates sorting

In [11]:
f = Freq_meth_calculate(
    input_fn="./data/freq_meth_calculate/methylation_calls.tsv",
    fasta_index="./data/freq_meth_calculate/ref.fa.fai",
    output_bed_fn="./output/freq_meth_calculate/out_freq_meth_calculate.bed",
    sample_id="Sample1",
    verbose=True)

head("./output/freq_meth_calculate/out_freq_meth_calculate.bed")

## Options summary ##
	package_name: NanopolishComp
	package_version: 0.6.9
	timestamp: 2019-08-30 18:14:36.578779
	quiet: False
	verbose: True
	sample_id: Sample1
	min_depth: 10
	output_tsv_fn: 
	output_bed_fn: ./output/freq_meth_calculate/out_freq_meth_calculate.bed
	fasta_index: ./data/freq_meth_calculate/ref.fa.fai
	input_fn: ./data/freq_meth_calculate/methylation_calls.tsv
## Checking arguments ##
	Testing input file readability
	Check output file
		Output results in bed format
## Parsing methylation_calls file ##
	Write output file header
	Starting to parse file Nanopolish methylation call file
	: 605248 lines [00:15, 39231.62 lines/s]
	Filtering out low coverage sites
	Sorting by coordinates
	Processing valid sites found
	: 100%|██████████| 999/999 [00:01<00:00, 673.84 sites/s] 
## Results summary ##
	Total read lines: 605,248
	Valid read lines: 605,248
	Total sites: 340,081
	Low coverage sites: 339,082
	Valid sites: 1,998


track name=Methylation_Sample1 itemRgb=On
chr-VIII	213351	213352	999	-0.500000	-	213351	213352	'100,100,100''
chr-VIII	213382	213386	1000	-5.103000	-	213382	213386	'8,121,207''
chr-VIII	213427	213428	1001	0.504545	-	213427	213428	'100,100,100''
chr-VIII	213516	213517	1002	-2.447500	-	213516	213517	'8,121,207''
chr-VIII	213536	213537	1003	-3.103333	-	213536	213537	'8,121,207''
chr-VIII	213549	213550	1004	-2.724167	-	213549	213550	'8,121,207''
chr-VIII	213645	213646	1005	-2.546667	-	213645	213646	'8,121,207''
chr-VIII	213668	213669	1006	-3.728333	-	213668	213669	'8,121,207''
chr-VIII	213729	213730	1007	-3.253846	-	213729	213730	'8,121,207''



#### Changing filtering threshold (not recommended)

In [7]:
f = Freq_meth_calculate(
    input_fn="./data/freq_meth_calculate/methylation_calls.tsv",
    output_tsv_fn="./output/freq_meth_calculate/out_freq_meth_calculate.tsv",
    min_llr=1,
    min_depth=20,
    min_meth_freq=0.3)

head("./output/freq_meth_calculate/out_freq_meth_calculate.tsv")

## Checking arguments ##
## Parsing methylation_calls file ##
	Starting to parse file Nanopolish methylation call file
	Processing_valid site found
## Results summary ##
	total read lines: 605,248
	Total sites: 340,081
	Low coverage sites: 339,249
	Low methylation sites: 821
	Valid sites: 11


chromosome start  end    strand site_id methylated_reads unmethylated_reads ambiguous_reads sequence       num_motifs meth_freq 
chr-XII    455556 455557 +      1089    16               6                  20              AGATCCGTTGT    1          0.380952  
chr-XII    461541 461542 +      1234    26               3                  52              AATTCCGAGGG    1          0.320988  
chr-XII    462606 462607 +      1266    21               9                  28              AATTCCGGGGT    1          0.362069  
chr-XII    464693 464694 +      1327    15               4                  15              AGATCCGTTGT    1          0.441176  
chr-XII    454376 454377 -      1418    10               7                  16              TCTTTCGGGTC    1          0.303030  
chr-XII    456198 456202 -      1464    17               13                 16              CAGCACGACGGAGT 2          0.369565  
chr-XII    458208 458209 -      1516    25               15                 37              TTAAA